In [50]:
import pandas as pd
import numpy as np
import pendulum
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os

In [51]:
from alpaca.data.historical.stock import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest 
from alpaca.data.timeframe import TimeFrame
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import GetAssetsRequest
from alpaca.trading.enums import AssetStatus
from alpaca.trading.enums import AssetClass
from alpaca.data.enums import Adjustment
from alpaca.data.timeframe import TimeFrameUnit

In [52]:
# Environment Variables
load_dotenv()

api_key = os.getenv("ALPACA_API_KEY")
secret_key = os.getenv("ALPACA_SECRET_KEY")

In [53]:
# clients
data_client = StockHistoricalDataClient(api_key,secret_key)
trading_client = TradingClient(api_key,secret_key)

In [54]:
# Parameters
end = pendulum.datetime(2024,7,4)
start = end.subtract(days=2)

In [55]:
# Get all current assets
asset_request = GetAssetsRequest(
    status = AssetStatus.ACTIVE,
    asset_class = AssetClass.US_EQUITY,
)

all_assets = trading_client.get_all_assets(asset_request)

all_assets = [vars(asset) for asset in all_assets] # Convert objects to dicts

In [56]:
df = pd.DataFrame(data=all_assets)

df = df[df['tradable'] & df['fractionable'] & df['shortable']]

df.head(1)

,id,asset_class,exchange,symbol,name,status,tradable,marginable,shortable,easy_to_borrow,fractionable,min_order_size,min_trade_increment,price_increment,maintenance_margin_requirement,attributes
474,053c672d-ee2a-4344-8907-3ad4d1d1e180,AssetClass.US_EQUITY,AssetExchange.NYSE,HGTY,"Hagerty, Inc.",AssetStatus.ACTIVE,True,True,True,True,True,None,None,None,30.0,"[fractional_eh_enabled, has_options]"


In [57]:
symbols = df['symbol'].to_list()

In [58]:
# Get previous days market data
bars_request = StockBarsRequest(
    symbol_or_symbols=symbols,
    timeframe=TimeFrame(1,TimeFrameUnit.Day),
    start=start,
    end=end,
    adjustment=Adjustment.ALL,
)

bars = data_client.get_stock_bars(bars_request)

In [59]:
df = bars.df

df = df.reset_index()

df.head(3)

,symbol,timestamp,open,high,low,close,volume,trade_count,vwap
0,ALTI,2024-07-02 04:00:00+00:00,4.86,4.900,4.6000,4.80,112836.0,1584.0,4.762368
1,ALTI,2024-07-03 04:00:00+00:00,4.84,4.840,4.6737,4.76,19077.0,467.0,4.746862
2,AWI,2024-07-02 04:00:00+00:00,110.95,112.985,110.9000,112.23,267223.0,6290.0,112.111249
